In [1]:
import os
from IPython.display import display, Markdown
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from getpass import getpass
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq

def md(t):
  display(Markdown(t))

/Users/muyang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [21]:
loader_OM = PyPDFLoader("./2024-Kona-SX2-OM.pdf")
loader_QRG = PyPDFLoader("./2024-KONA-SX2-QRG.pdf")
pages_OM = loader_OM.load()
pages_QRG = loader_QRG.load()
all_pages = pages_OM + pages_QRG

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
all_splits = text_splitter.split_documents(all_pages)

vectorstore = FAISS.from_documents(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever()

/Users/muyang/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
len(all_splits)

802

In [23]:
GROQ_API_TOKEN = getpass()
# GROQ_API_TOKEN = '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'
os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [24]:
llm_result = llm.invoke("How to turn the rear window wiper?")
md(llm_result.content)

The rear window wiper is a convenient feature that helps keep your rear window clean and clear of debris, especially during rainy or snowy weather. Here's how to turn it on:

**Method 1: Using the Wiper Stalk**

1. Locate the wiper stalk, usually located on the steering column or on the center console.
2. Pull the stalk towards you to activate the rear wiper.
3. You may need to rotate the stalk or press a button to switch between intermittent and continuous modes.

**Method 2: Using a Button or Switch**

1. Check your vehicle's dashboard or center console for a button or switch labeled "Rear Wiper" or "Rear Washer."
2. Press the button or flip the switch to turn the rear wiper on or off.
3. Some vehicles may have a separate button for the rear wiper, while others may have a combined button for both front and rear wipers.

**Method 3: Using the Infotainment System (if equipped)**

1. If your vehicle has an infotainment system, you may be able to control the rear wiper through the touchscreen display.
2. Navigate to the "Settings" or "Vehicle" menu and look for the "Rear Wiper" or "Rear Washer" option.
3. Select the option to turn the rear wiper on or off.

Remember to check your vehicle's owner's manual if you're unsure about the specific procedure for your vehicle.

In [30]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)



### Answer question ###
qa_system_prompt = """You are an expert to answer drivers question related to their vehicle. \
Your knowledge base is the following pieces of retrieved context. \
Only answer the question based on this provided information. \
If you don't know the answer, just say that you don't know. \
Keep the answer concise, don't start with 'ccording to the provided information',\
{context}"""


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [31]:
def auto_chat(input, user_id, show_reference = False):
    result = conversational_rag_chain.invoke(
        {"input": input},
        config={
            "configurable": {"session_id": user_id}
        }
    )
    md(result["answer"])
    if show_reference:
        print("Reference:")
        for i in range(len(result["context"])):
            print(result["context"][i].metadata)
            print(result["context"][i].page_content)
            print("-"*100, "\n\n")

In [32]:
auto_chat("Does my car has spare tire?", "Muyang", True)

Parent run 7aaab75d-83a4-4efb-b98e-c4eb1e204511 not found for run a24844c3-7079-4ac2-8bce-ecbc74dc165d. Treating as a root run.


Yes, your car has a spare tire.

Reference:
{'source': './2024-Kona-SX2-OM.pdf', 'page': 479}
8
8-13If You Have A Flat Tire 
(With Spare Tire)
 WARNING
 
Follow the instructions in this section 
when replacing a tire to reduce the risk of serious injury or death. Changing a tire can be dangerous.
 
Jack and tools
2C_SpareTireReplacementOverview
(1) Jack handle
(2) Jack(3) Wheel lug wrench
The jack, jack handle, and wheel lug nut 
wrench are stored in the cargo area under the luggage box cover.
The jack is provided for emergency tire 
changing only.
2C_SpareTireReplacementPrecedure
Turn the winged hold down bolt counterclockwise to remove the spare tire.Store the spare tire in the same compartment by turning the winged hold down bolt clockwise.
To prevent the spare tire and tools from 
"rattling", store them in their proper locations.
2C_SpareTireReplacementPrecedure_2
If it is hard to loosen the tire hold down wing bolt by hand, you can loosen it easily using the wheel lug wrench.
Turn the tire hold down wing bolt 
co

In [33]:
auto_chat("How do I install it? Provide me with detailed steps", "Muyang", True)

Parent run 4f20956a-b9d3-4649-87e2-de9c262137f6 not found for run 227f6652-7714-4a74-9c18-35969839d7a5. Treating as a root run.


Here are the steps to install the spare tire:

**Step 1: Locate the spare tire and tools**

* The spare tire, jack, jack handle, and wheel lug wrench are stored in the cargo area under the luggage box cover.

**Step 2: Remove the spare tire**

* Turn the winged hold down bolt counterclockwise to remove the spare tire.

**Step 3: Loosen the flat tire**

* Use the wheel lug wrench to loosen the flat tire by turning it counterclockwise.

**Step 4: Raise the vehicle**

* Use the jack to raise the vehicle. Make sure to follow the safety precautions:
	+ NEVER place any portion of your body under the vehicle that is supported by a jack.
	+ ONLY use the jack provided with the vehicle.

**Step 5: Remove the flat tire**

* Completely remove the flat tire and replace it with the spare tire.

**Step 6: Tighten the spare tire**

* Use the wheel lug wrench to tighten the spare tire by turning it clockwise.

**Step 7: Lower the vehicle**

* Carefully lower the vehicle to the ground using the jack.

**Step 8: Tighten the spare tire again**

* Once the vehicle is back on the ground, use the wheel lug wrench to tighten the spare tire as tight as possible in a star pattern.

Remember to drive carefully and safely to a repair shop to have the flat tire repaired or replaced as soon as possible.

Reference:
{'source': './2024-Kona-SX2-OM.pdf', 'page': 467}
Jack and tools ............................................................................................................ 8-13Changing tires ............................................................................................................ 8-13Jack label.................................................................................................................... 8-18
If You Have A Flat Tire (with Tire Mobility Kit)............................................................. 8-19
Introduction................................................................................................................ 8-19Notes on the safe use of the Tire Mobility Kit..........................................................8-20Components of the Tire Mobility Kit .........................................................................8-21Using the Tire Mobility Kit when a tire is flat ......................................

In [34]:
auto_chat("How to connect to carplay", "Muyang", True)

Parent run 60b2f1fe-7fd9-4d23-8508-51c278e3baf8 not found for run 706f2f55-9ed2-464c-a5e4-1f8029abf110. Treating as a root run.


To connect to CarPlay, follow these steps:

**Phone Setup:**

1. On your iPhone, go to Settings > General > CarPlay.
2. Ensure Siri, Wi-Fi, and Bluetooth are turned ON.

**Car Setup:**

1. Activate Wireless Connectivity on your car's infotainment system.
2. On the Home Screen, select All Menus > Setup > Wi-Fi > Use Wi-fi Connection.
3. Connect to CarPlay on the infotainment system.

**Pairing:**

1. On your iPhone, search for available Wi-Fi networks and select your car's Wi-Fi network.
2. On the infotainment system, select All Menus > Setup > Device Connections > Phone Projection > Add New.
3. Follow the prompts on the infotainment system and your iPhone to complete the pairing process.

**Start Using CarPlay:**

1. On the infotainment system, select the CarPlay button.
2. You will see all the CarPlay-supported apps on the infotainment system.

Note: Ensure your iPhone is running iOS 10 or higher, and your car's infotainment system is compatible with CarPlay.

Reference:
{'source': './2024-KONA-SX2-QRG.pdf', 'page': 15}
INFOTAINMENT Quick Reference Guide I 28 27 I Quick Reference Guide
KONA HYUNDAI ON YOUR SMARTPHONE
- Activate Bluetooth and Wifi
- Search for your vehicle on Bluetooth 
device list
- Select your Vehicle
- Confirm Bluetooth passkeys match 
the touch screen display
- Read and accept prompts from the 
touch screen display
PHONE SETUP 
1. On your phone, go to Settings > General > 
Carplay to allow Carplay access.
2. On your phone, go to Settings > Siri, Wi-Fi 
and Bluetooth and ensure they are turned 
ON.REQUIREMENTS:
- iOS software 10 and higher 
- Compatible iOS Phone that supports 
Wireless Carplay
- Data and Wireless plan for applicable 
features
BEFORE YOU BEGIN:
- Carplay is only available via Wi-Fi 
Phone Projection
CAR SETUP
Activate Wireless Connectivity
On the Home Screen, select the following:
All Menus > Setup > Wi-Fi > Use Wi-fi 
ConnectionConnect to CarPlay
On the Home Screen, select the following: All Menus > CarPl